# Obtenção e organização dos dados

In [1]:
import pandas as pd

df1 = pd.read_csv('../dados/acordaos-unicos.csv', sep = '|')[['acordao', 'areas', 'texto']]
df1.head()

,acordao,areas,texto
0,297/2016-P,Responsabilidade,TRIBUNAL DE CONTAS DA UNIÃO\tTC 010.084/2015-0...
1,366/2016-P,Finanças Públicas,TRIBUNAL DE CONTAS DA UNIÃO\tTC 005.933/2014-5...
2,944/2016-P,Responsabilidade,TRIBUNAL DE CONTAS DA UNIÃO\tTC 042.038/2012-0...
3,30/2016-P,Direito Processual,TRIBUNAL DE CONTAS DA UNIÃO\tTC 000.742/2014-7...
4,55/2016-P,Pessoal,;-;;Wania Lucia Pasquarelli do NascimentoTCUWa...


In [2]:
df2 = pd.read_csv('../dados/acordaos-unicos-filtrados-6000.csv', sep = '|')[['acordao', 'filtrado_6000']]
df2['filtrado_6000'] = df2['filtrado_6000'].astype(str)
df2.head()

,acordao,filtrado_6000
0,297/2016-P,tribunal conta união tc grupo classe plenário ...
1,366/2016-P,tribunal conta união tc grupo classe ii plenár...
2,944/2016-P,tribunal conta união tc grupo classe plenário ...
3,30/2016-P,tribunal conta união tc grupo classe plenário ...
4,55/2016-P,wania lucia pasquarelli nascimentotcuwania luc...


In [3]:
df = df1.merge(df2, on='acordao', how='inner')
df.head()

,acordao,areas,texto,filtrado_6000
0,297/2016-P,Responsabilidade,TRIBUNAL DE CONTAS DA UNIÃO\tTC 010.084/2015-0...,tribunal conta união tc grupo classe plenário ...
1,366/2016-P,Finanças Públicas,TRIBUNAL DE CONTAS DA UNIÃO\tTC 005.933/2014-5...,tribunal conta união tc grupo classe ii plenár...
2,944/2016-P,Responsabilidade,TRIBUNAL DE CONTAS DA UNIÃO\tTC 042.038/2012-0...,tribunal conta união tc grupo classe plenário ...
3,30/2016-P,Direito Processual,TRIBUNAL DE CONTAS DA UNIÃO\tTC 000.742/2014-7...,tribunal conta união tc grupo classe plenário ...
4,55/2016-P,Pessoal,;-;;Wania Lucia Pasquarelli do NascimentoTCUWa...,wania lucia pasquarelli nascimentotcuwania luc...


In [4]:
df1.shape, df2.shape, df.shape

((9739, 3), (9739, 2), (9739, 4))

# Pré-processamento

In [5]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
y = lbArea.transform(df['areas'])
lbArea.classes_, y.shape

(array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
        'Desestatização', 'Direito Processual', 'Finanças Públicas',
        'Gestão Administrativa', 'Licitação', 'Pessoal',
        'Responsabilidade'], dtype='<U23'), (9739, 10))

In [7]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

print('\tCarregamento do modelo de acordaos...')
modelo = Word2Vec.load('../vocabularios/modelo-acordaos-50.w2v')

	Carregamento do modelo de acordaos...


/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
from keras.preprocessing.text import Tokenizer
import numpy as np

df_scores = pd.DataFrame()

limite_texto = 6000
dim_vetor = 50

print('\tTokenizacao e montagem de sequencias...')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['filtrado_6000'])
vocabulario = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(df['filtrado_6000'])

print('\tMontagem da matriz de embeddings...')
embedding_matrix = np.zeros((vocabulario, dim_vetor))
for word, i in tokenizer.word_index.items():
    if word in modelo.wv:
        embedding_matrix[i] = modelo.wv[word]

Using TensorFlow backend.


	Tokenizacao e montagem de sequencias...
	Montagem da matriz de embeddings...


In [10]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)
(x.shape, y.shape)

((9739, 6000), (9739, 10))

# Predição Modelo Recorrente

In [11]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, GlobalMaxPooling1D, Flatten, GRU
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=True,  weights=[embedding_matrix]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=['categorical_accuracy'])

model.load_weights('../rascunho/modelos-acordaos/weights0.hdf5')
y_pred = model.predict_classes(x, verbose=1)
y_pred

W0322 19:47:29.661564 140079647733568 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0322 19:47:29.673598 140079647733568 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0322 19:47:29.675956 140079647733568 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0322 19:47:29.683027 140079647733568 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

9739/9739 [==============================] - 540s 55ms/step


array([9, 5, 4, ..., 8, 9, 9])

In [13]:
from sklearn.metrics import classification_report

y_i = [list(x).index(1) for x in y]
report = classification_report(y_i, y_pred, target_names=lbArea.classes_, output_dict = True)
report

{'Competência do TCU': {'precision': 0.7142857142857143,
  'recall': 0.037037037037037035,
  'f1-score': 0.0704225352112676,
  'support': 405},
 'Contrato Administrativo': {'precision': 0.6127946127946128,
  'recall': 0.331511839708561,
  'f1-score': 0.43026004728132383,
  'support': 549},
 'Convênio': {'precision': 0.6818181818181818,
  'recall': 0.09433962264150944,
  'f1-score': 0.16574585635359118,
  'support': 477},
 'Desestatização': {'precision': 0.6491228070175439,
  'recall': 0.45121951219512196,
  'f1-score': 0.5323741007194245,
  'support': 82},
 'Direito Processual': {'precision': 0.5734126984126984,
  'recall': 0.5995850622406639,
  'f1-score': 0.5862068965517241,
  'support': 1446},
 'Finanças Públicas': {'precision': 0.34203655352480417,
  'recall': 0.6582914572864321,
  'f1-score': 0.45017182130584193,
  'support': 199},
 'Gestão Administrativa': {'precision': 0.8333333333333334,
  'recall': 0.19230769230769232,
  'f1-score': 0.3125,
  'support': 130},
 'Licitação': {'p

# Preparação dos resultados

In [14]:
y_pred

array([9, 5, 4, ..., 8, 9, 9])

In [15]:
lbArea.classes_[y_pred]

array(['Responsabilidade', 'Finanças Públicas', 'Direito Processual', ...,
       'Pessoal', 'Responsabilidade', 'Responsabilidade'], dtype='<U23')

In [16]:
pd.DataFrame(lbArea.classes_[y_pred]).head()

,0
0,Responsabilidade
1,Finanças Públicas
2,Direito Processual
3,Direito Processual
4,Pessoal


In [20]:
df_preds = df[['acordao', 'areas']]

In [22]:
df_preds['preds_recor_filtrados'] = lbArea.classes_[y_pred]

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_preds.head()

,acordao,areas,preds
0,297/2016-P,Responsabilidade,Responsabilidade
1,366/2016-P,Finanças Públicas,Finanças Públicas
2,944/2016-P,Responsabilidade,Direito Processual
3,30/2016-P,Direito Processual,Direito Processual
4,55/2016-P,Pessoal,Pessoal


In [27]:
len(df_preds.query('areas == preds_recor_filtrados')), len(df_preds), len(df_preds.query('areas == preds_recor_filtrados')) / len(df_preds)

(6963, 9739, 0.7149604682205565)

# Predição Modelo Convolucional Filtrado

In [30]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

modelo = Word2Vec.load('../vocabularios/modelo-acordaos.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [31]:
dim_vetor = 100
limite_texto = 6000

print('\tTokenizacao e montagem de sequencias...')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['filtrado_6000'])
vocabulario = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(df['filtrado_6000'])

print('\tMontagem da matriz de embeddings...')
embedding_matrix = np.zeros((vocabulario, dim_vetor))
for word, i in tokenizer.word_index.items():
    if word in modelo.wv:
        embedding_matrix[i] = modelo.wv[word]

	Tokenizacao e montagem de sequencias...
	Montagem da matriz de embeddings...


In [32]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=True,  weights=[embedding_matrix]))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())

model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=['categorical_accuracy'])

model.load_weights('../rascunho/modelos-acordaos/weights1-0.hdf5')
y_pred = model.predict_classes(x, verbose=1)
y_pred

9739/9739 [==============================] - 4s 402us/step


array([9, 5, 9, ..., 8, 9, 4])

In [33]:
df_preds['preds_conv_filtrados'] = lbArea.classes_[y_pred]

In [37]:
df_preds.head()

,acordao,areas,preds_conv_filtrados,preds_recor_filtrados
0,297/2016-P,Responsabilidade,Responsabilidade,Responsabilidade
1,366/2016-P,Finanças Públicas,Finanças Públicas,Finanças Públicas
2,944/2016-P,Responsabilidade,Responsabilidade,Direito Processual
3,30/2016-P,Direito Processual,Direito Processual,Direito Processual
4,55/2016-P,Pessoal,Pessoal,Pessoal


In [39]:
len(df_preds.query('areas == preds_conv_filtrados')) / len(df_preds), len(df_preds.query('areas == preds_recor_filtrados')) / len(df_preds)

(0.9333607146524284, 0.7149604682205565)

In [41]:
len(df_preds.query('areas == preds_conv_filtrados or areas == preds_recor_filtrados')) / len(df_preds)

0.9561556627990554

# Predição Modelo Convolucional Original

In [42]:
dim_vetor = 100
limite_texto = 40000

print('\tTokenizacao e montagem de sequencias...')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['texto'])
vocabulario = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(df['texto'])

print('\tMontagem da matriz de embeddings...')
embedding_matrix = np.zeros((vocabulario, dim_vetor))
for word, i in tokenizer.word_index.items():
    if word in modelo.wv:
        embedding_matrix[i] = modelo.wv[word]

	Tokenizacao e montagem de sequencias...
	Montagem da matriz de embeddings...


In [45]:
x = pad_sequences(sequences, maxlen=limite_texto)
(x.shape, y.shape)

((9739, 40000), (9739, 10))

In [46]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=True,  weights=[embedding_matrix]))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())

model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=['categorical_accuracy'])

model.load_weights('../rascunho/modelos-acordaos/weights0-0.hdf5')
y_pred = model.predict_classes(x, verbose=1)
y_pred

9739/9739 [==============================] - 18s 2ms/step


array([9, 5, 9, ..., 8, 9, 4])

In [47]:
df_preds['preds_conv_originais'] = lbArea.classes_[y_pred]

In [48]:
df_preds.head()

,acordao,areas,preds_conv_filtrados,preds_recor_filtrados,preds_conv_originais
0,297/2016-P,Responsabilidade,Responsabilidade,Responsabilidade,Responsabilidade
1,366/2016-P,Finanças Públicas,Finanças Públicas,Finanças Públicas,Finanças Públicas
2,944/2016-P,Responsabilidade,Responsabilidade,Direito Processual,Responsabilidade
3,30/2016-P,Direito Processual,Direito Processual,Direito Processual,Direito Processual
4,55/2016-P,Pessoal,Pessoal,Pessoal,Pessoal


In [49]:
len(df_preds.query('areas == preds_conv_originais')) / len(df_preds), len(df_preds.query('areas == preds_conv_filtrados')) / len(df_preds), len(df_preds.query('areas == preds_recor_filtrados')) / len(df_preds)

(0.9266865181230106, 0.9333607146524284, 0.7149604682205565)

In [50]:
len(df_preds.query('areas == preds_conv_originais or areas == preds_conv_filtrados or areas == preds_recor_filtrados')) / len(df_preds)

0.9722764144162646

In [51]:
df_preds.to_csv('acordaos-unicos-preds.csv', encoding = 'Latin1')

In [58]:
from sklearn import tree
X = np.array([lbArea.transform(df_preds['preds_conv_originais']), lbArea.transform(df_preds['preds_conv_originais']), lbArea.transform(df_preds['preds_recor_filtrados'])])
Y = lbArea.transform(df_preds['areas'])
X.shape

(3, 9739, 10)

In [67]:
def cod(cc):
    for i, c in enumerate(lbArea.classes_):
        if c == cc:
            return i
    return -1

In [69]:
df_preds['preds_conv_originais'].apply(cod).values

array([9, 5, 9, ..., 8, 9, 4])

In [82]:
import numpy as np

X = np.array([df_preds['preds_conv_originais'].apply(cod).values, df_preds['preds_conv_filtrados'].apply(cod).values, df_preds['preds_recor_filtrados'].apply(cod).values]).T
X

array([[9, 9, 9],
       [5, 5, 5],
       [9, 9, 4],
       ...,
       [8, 8, 8],
       [9, 9, 9],
       [4, 4, 9]])

In [83]:
Y = df_preds['areas'].apply(cod).values
Y

array([9, 5, 9, ..., 8, 9, 4])

In [84]:
X.shape

(9739, 3)

In [85]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [87]:
df_preds['preds_stack'] = lbArea.classes_[clf.predict(X)]

In [88]:
df_preds.head()

,acordao,areas,preds_conv_filtrados,preds_recor_filtrados,preds_conv_originais,preds_stack
0,297/2016-P,Responsabilidade,Responsabilidade,Responsabilidade,Responsabilidade,Responsabilidade
1,366/2016-P,Finanças Públicas,Finanças Públicas,Finanças Públicas,Finanças Públicas,Finanças Públicas
2,944/2016-P,Responsabilidade,Responsabilidade,Direito Processual,Responsabilidade,Responsabilidade
3,30/2016-P,Direito Processual,Direito Processual,Direito Processual,Direito Processual,Direito Processual
4,55/2016-P,Pessoal,Pessoal,Pessoal,Pessoal,Pessoal


In [89]:
len(df_preds.query('areas == preds_stack')) / len(df_preds)

0.9491734264298183

In [90]:
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [93]:
from sklearn.tree.export import export_text

print(export_text(clf, feature_names=['preds_conv_filtrados', 'preds_recor_filtrados', 'preds_conv_originais']))

|--- preds_conv_filtrados <= 7.50
|   |--- preds_conv_filtrados <= 6.50
|   |   |--- preds_conv_filtrados <= 3.50
|   |   |   |--- preds_conv_filtrados <= 1.50
|   |   |   |   |--- preds_recor_filtrados <= 0.50
|   |   |   |   |   |--- preds_conv_filtrados <= 0.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- preds_conv_filtrados >  0.50
|   |   |   |   |   |   |--- preds_conv_originais <= 8.00
|   |   |   |   |   |   |   |--- preds_conv_originais <= 4.50
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- preds_conv_originais >  4.50
|   |   |   |   |   |   |   |   |--- preds_conv_originais <= 6.00
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- preds_conv_originais >  6.00
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- preds_conv_originais >  8.00
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- preds_recor_filtrados >  0.50
|   |   |   |   |   |--- preds_r